<h1 align="center"> Python for the Humanities and Social Sciences <br> *Data Manipulation* </h1>

## Info
- Scott Bailey (CIDR), *scottbailey@stanford.edu*
- Javier de la Rosa (CIDR), *versae@stanford.edu*
- Ashley Jester (CIDR/SSDS), *ajester@stanford.edu*
- Green Library 121A, 2pm-4pm

## Topics
- Pandas Series and DataFrame (numpy, scipy)
- Loading data in, null and missing data
- Describing data
- Column manipulation
- String manipulation (super basic regex)
- Split-Apply-Combine
- Plotting (matplotlib, seaborn):
  - Basic charts (line, bar, pie)
  - Histograms
  - Scatter plots
  - Boxplots, violinplots

### Virtual environments (venvs) and Anaconda
- Isolated environment so each project can have its own dependencies without conflicts with other projects
- Anaconda has its own environment manager and package manager, let's you easily set Python versions, and comes with many of the standard packages used in scientific computing

To set up the environment for this project, in your BASH shell, run (`$` means a shell command):

```
$ conda create -n data python=3.5 anaconda seaborn
```

Or installing the specific packages we'll be using:

```
$ conda create -n data python=3.5
$ conda install -n data jupyter pandas numpy scipy matplotlib seaborn requests
```

This creates an environment named `data`, where the python version is specified to 3.5, and installs the necessary packages for data exploration and manipulation in the environment.

After you create the environment, run `source activate data` or `activate data` depending on whether you are on OSX or Windows to activate the environment.

### Jupyter Notebooks
- Used to be IPython Notebooks
- Write and evaluate code at a granular level without rerunning scripts constantly and using a lot of print debugging
- Mix in Markdown and HTML within your notebook, and so is a great way of presenting code and data analysis

Once you have a virtual environment running, just run `jupyter notebook` from the location where you want to store your notebook.

```
$ jupyter notebook
```

And go to http://localhost:8888/ in your browser.

## Pandas

From Jake Vanderplas' book [**Python Data Science Handbook**](http://shop.oreilly.com/product/0636920034919.do) (from which some code excerpts are used in this workshop):

> Pandas is a newer package built on top of NumPy, and provides an efficient implementation of a `DataFrame`. `DataFrame`s are essentially multidimensional arrays with attached row and column labels, and often with heterogeneous types and/or missing data. As well as offering a convenient storage interface for labeled data, Pandas implements a number of powerful data operations familiar to users of both database frameworks and spreadsheet programs.

In [ ]:
import numpy as np
import pandas as pd
import requests

# Set some options
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 10)

There are three main data structures in Pandas: `Series`, `DataFrame`, and `Index`. Pandas has a very decent [documentation](http://pandas.pydata.org/pandas-docs/stable/), and using Jupyter, any method help can be shown by appending the a `?` to the end and running the cell.

In [ ]:
# For example
pd.isnull?

### `Series`
A `Series` is a one-dimensional array of indexed data. It can be seens as a specialized dictionary or a generalized NumPy array.

In [ ]:
pd.Series([1, 2, 3, 4])

In [ ]:
pd.Series([1, 2, 3, 4], index=["a", "b", "c", "d"])

In [ ]:
pd.Series({"a": 1, "b": 2, "c": 3, "d": 4})

In [ ]:
pd.Series([1, 2, 3, 4], index=["a", "b", "c", "d"]) == pd.Series({"a": 1, "b": 2, "c": 3, "d": 4})

In [ ]:
# Accessing elements
s = pd.Series([1, 2, 3, 4], index=["a", "b", "c", "d"])
s["a"]

In [ ]:
s["b":"d"]

In [ ]:
# NumPy array underneath
s.values

In [ ]:
s.sum()

In [ ]:
s.prod()

In [ ]:
# With an index
s.index

### `DataFrame`

A `DataFrame` is a two-dimensional array with both flexible row indices and flexible column names. It can be seen as 
as a generalization of a two-dimensional NumPy array, or a specialization of a dictionary in which each column name maps to a `Series` of column data.

In [ ]:
population_dict = {'California': 38332521, 'Texas': 26448193, 'New York': 19651127,
                   'Florida': 19552860, 'Illinois': 12882135}
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
states = pd.DataFrame({'population': population_dict, 'area': area_dict})
states

In [ ]:
population = pd.Series(population_dict)
area = pd.Series(area_dict)
pd.DataFrame({'population': population, 'area': area})

In [ ]:
# Index
states.index

In [ ]:
# Values
states.values

In [ ]:
# Columns
states.columns

`DataFrames` can be created in different ways

In [ ]:
# From regular dictionaries
data = {'column A': [1,2,3,4], 'column B': list('abcd')}
pd.DataFrame(data)

In [ ]:
pd.DataFrame.from_dict(data)

In [ ]:
# From lists
data = [(1,'a'),(2,'b'),(3,'c'),(4,'d')]
labels = ['ColumnA','ColumnB']
pd.DataFrame.from_records(data, columns=labels)

### Index

In [ ]:
states.index

In [ ]:
# Acts like a Python set() that supports duplicated items
"California" in states.index

## Data I/O

Pandas provides a few methods to load in and out data in CSVs, Excel spreadsheets, HDF, or even JSON format.

For example, click in the next URL of a CSV file containing twitter data during the release of the Apple Watch: http://bit.ly/python_workshop_data

In [ ]:
# Pandas can even fetch data from a URL
pd.read_csv("http://bit.ly/python_workshop_data")

Let's save the previous data to a locala file.

In [ ]:
with open("twitter.csv", "wb") as file:
    file.write(requests.get("http://bit.ly/python_workshop_data").content)

In [ ]:
df = pd.read_csv("twitter.csv")
df

In [ ]:
# Showing the columns
df.columns

Let's reload the CSV but this time specifying a index column

In [ ]:
pd.read_csv("twitter.csv", index_col="created_at")

Let's also filter out some columns we are not interested, provide data types for a couple, and show the first 5.

In [ ]:
columns = [
    "created_at", "id", "lang", 
    "place", "possibly_sensitive", "text",
    "user_screen_name", "user_name", "user_lang", "user_location",
    "hashtags", "media", "symbols", "urls", "lat", "lon", "country"]
index = "created_at"
data_types = {
    "id": int,
    "possibly_sensitive": bool,
    "lat": float,
    "lon": float,
}
df = pd.read_csv("twitter.csv", parse_dates=["created_at"], index_col="created_at", usecols=columns, dtype=data_types)
df.head(5)

In [ ]:
# Let's find out the data types
df.dtypes

Finally we can get rid of those ugly `NaN`s (which is the Pandas way of telling that not valid data has been found in a cell). We'll first drop rows with just `NaN`s and then fill those of type string with am empty string.

In [ ]:
nans = {col: "" for col in ["lang", "place", "text", "user_screen_name", "user_name", "user_lang",
                            "user_location", "hashtags", "media", "symbols", "urls", "country"]}
df.fillna(value=nans, inplace=True)
df.head(5)

In [ ]:
previous_count = df.count()  # .count() basically counts elements
df = df.dropna(subset=["id"]).dropna(how="all")

We can also remove duplicate rows based on all cell content or individual columns.

In [ ]:
df = df.drop_duplicates(subset=["id"]).drop_duplicates()

Let's see the difference

In [ ]:
previous_count.id, df.count().id

Now we can just save the clean data to any format supported by Pandas

In [ ]:
df.to_csv("twitter_clean.csv", encoding="utf8")

## Indexing and selecting

Accessing column data

In [ ]:
states.population

In [ ]:
states["population"]

In [ ]:
# We can create a "mask"
states.population > 3e7

In [ ]:
# And filter based on any logical expression
states[states.population > 3e7]

Also with *fancy* indexing

In [ ]:
states[["population"]]

In [ ]:
type(states["population"]), type(states[["population"]])

In [ ]:
# Area of states with population higher than 30000000 people
states[states.population > 3e7][["area"]]

In [ ]:
# It's also possible to access individual row data
states.iloc[0]

In [ ]:
states.loc["California"]

In [ ]:
states.ix["California", "area"]

## Manipulation

One of the most basic operations you can do with data is counting. Let's try to get the how many times each hashtag is present in the twitter dataset.

In [ ]:
df[["hashtags"]].dropna()

Using string operations we can `.split()` by comma and get a list of hashtags.

In [ ]:
df.hashtags.dropna().str.split(",")

In [ ]:
# Equivalent to
df.hashtags.dropna().apply(lambda x: x.split(","))

We can now *sum* all those lists together and create a `Series` with the that.

In [ ]:
df.hashtags.dropna().str.split(",").sum()[:10]

In [ ]:
hashtags = pd.Series(df.hashtags.dropna().str.split(",").sum())
hashtags

One handy function of Pandas is `.value_counts()` which unsurprisingly counts how many times an element is present. 

In [ ]:
hashtags.value_counts()

Actually, this is not the fastest way to do this. Python's stantard library `collections` are out of the scope of this workshop, but its `Counter` class is commonly use for the same purpose.

In [ ]:
from collections import Counter
Counter(",".join(df.hashtags.dropna().values).split(",")).most_common(10)

The same pattern can be apply to `symbols`, `media`, or `urls`

In [ ]:
for column in ["symbols", "media", "urls"]:
    column_series = pd.Series(df[column].dropna().str.split(",").sum())
    print(column)
    print(column_series.value_counts()[:10])
    print()

### Grouping data

But what about the most tweeted language? Or the most prolific user? For this kind of operations we need to use what is called an [Split-Apply-Combine](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf) approach. In Pandas this can take the form of a `.groupby()` operation followed by an `.aggregate()` function.

In [ ]:
df.groupby("lang")

In [ ]:
df.groupby("lang")[["text"]]  # no computation is made yet!

In [ ]:
df.groupby("lang")[["text"]].aggregate(np.count_nonzero)

In [ ]:
# Sort by text
df.groupby("lang")[["text"]].aggregate(np.count_nonzero).sort_values("text", ascending=False)[:10]

Same pattern can be applied to `user_screen_name`, `place`, or `country`.

In [ ]:
def count_by(df, column, by="text", count=10):
    return (df.groupby(column)[[by]]
              .aggregate(np.count_nonzero).sort_values(by, ascending=False)[:count])

count_by(df, "user_screen_name")

In [ ]:
count_by(df[df.place.str.len() > 0], "place")

In [ ]:
count_by(df[df.country.str.len() > 0], "country")

However, for complex groupings like this a pivot table can be more useful.

In [ ]:
df.pivot_table(
    index=["lang", "user_screen_name"],
    values=["text"],
    aggfunc=np.count_nonzero
).sort_values("text", ascending=False)

And just out of curiosity, what's the average length of the tweets?

In [ ]:
df["len"] = df.text.apply(len)
df["len"].mean()

Let's now get the most popular retweet in the English language

In [ ]:
en_text = df[df['lang'] == 'en'][['text']]
en_retweets = en_text[en_text.text.str.startswith("RT @")]
count_by(en_retweets, "text", count=1)

## Visualization

There are several libraries to handle visualization of data in Python. `matplotlib` is probably the most widely used and also the most intricate to learn. For that reason some replacements and wrappers extending it have appeared over the years, the most prominent one being `seaborn`.

In [ ]:
# enables inline plotting in Jupyter
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
x = np.linspace(0, 10, 100)

fig, ax = plt.subplots(1, figsize=(15, 5))
ax.plot(x, np.sin(x))
ax.plot(x, np.cos(x))
# Besides this object-oriented paradigm, matplotlib also provides MATLAB-based syntax

There are other styles available as well.

In [ ]:
with plt.style.context('ggplot'):
    fig, ax = plt.subplots(1, figsize=(15, 5))
    ax.plot(x, np.sin(x))
    ax.plot(x, np.cos(x))

In [ ]:
plt.style.available

In [ ]:
# Even a special one for XKCD!
with plt.xkcd():
    fig, ax = plt.subplots(1, figsize=(15, 5))
    ax.plot(x, np.sin(x))
    ax.plot(x, np.cos(x))

`seaborn` changes the default style after being imported, but it can be reverted back easily setting the default style to `classic`.

In [ ]:
plt.style.use("seaborn")
fig, ax = plt.subplots(1, figsize=(15, 5))
ax.plot(x, np.sin(x))
ax.plot(x, np.cos(x))

Pandas also provides some utilities to create basic plots.

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 5))
count_by(df, "lang").plot(ax=ax,
    kind="bar",
)
ax.set_title("Languages")

Let's create a hitogram with the lengths of tweets.

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 5))
df["len"].hist(ax=ax, bins=15, normed=True, color='lightseagreen')
df["len"].plot(ax=ax, kind='kde', xlim=(0, 150), style='r--')
ax.set_title("Histogram of lengths of tweets")

In [ ]:
df.boxplot(column="len", grid=False)

Let's now try to find out if there is any sort of relationship between the length of a tweet and the number of hastags it uses.

In [ ]:
df["hashtags_count"] = df.hashtags.apply(lambda x: len(x.split(",")))
df[["len", "hashtags_count"]]

In [ ]:
fig, ax = plt.subplots(1, figsize=(15, 5))
ax.scatter(df.hashtags_count, df.len)
ax.set_ylabel("Length")
ax.set_xlabel("# Hashtags")
ax.set_title("Tweets length by number of hashtags")